In [2]:
import csv
import numpy as np
from matplotlib import pyplot
from sklearn import linear_model
from sklearn.model_selection import cross_validate
import pandas as pd
import datetime as dt
import math
import sys

In [3]:
#Sort the data in the file
#read a file
file = open('2018.3.13~2018.12.16_logger01.txt')
Allfile = file.readlines()
file.close()
Allfilelisted = [0]*len(Allfile)

#split each list at blanks
for n in range(len(Allfile)):
    Allfilelisted[n] = Allfile[n].split()
    
#extract title
Data = Allfilelisted[4:]
Datatitle = Allfilelisted[1][:]


In [4]:
#Start using pandas
df = pd.DataFrame(Data, dtype=np.float64)
#Put date and time together because I wanna let "to_datetime" read date and time later.
date_time = df[:][0] + " " + df[:][1]
#Let "to_datatime" read the date and time and convert them into datetime64
converted_time = pd.to_datetime(date_time)
# print(converted_time)
#insert this date&time data into the Data set(Delite the original date and time(Column 0 & 1))
df1 = pd.concat([converted_time, df.iloc[:, 2:]], axis=1)
#Put titles on the columns
df1.columns = Datatitle
print(df1)

                 TIMESTAMP    RECORD  Bat_Avg  Pit_WL_Avg  L_UR_Avg  L_UL_Avg  \
0      2018-03-13 16:23:00     135.0    12.71       11.55     57.07     11.91   
1      2018-03-13 16:24:00     136.0    12.71       11.56     57.07     11.91   
2      2018-03-13 16:25:00     137.0    12.71       11.57     57.01     11.91   
3      2018-03-13 16:26:00     138.0    12.72       11.58     57.06     11.91   
4      2018-03-13 16:27:00     139.0    12.72       11.57     57.06     11.91   
5      2018-03-13 16:28:00     140.0    12.72       11.59     57.05     11.91   
6      2018-03-13 16:29:00     141.0    12.72       11.58     57.04     11.91   
7      2018-03-13 16:30:00     142.0    12.72       11.59     57.05     11.91   
8      2018-03-13 16:31:00     143.0    12.72       11.61     57.05     11.91   
9      2018-03-13 16:32:00     144.0    12.72       11.60     57.00     11.91   
10     2018-03-13 16:33:00     145.0    12.72       11.61     57.03     11.91   
11     2018-03-13 16:34:00  

In [5]:
print(df1.dtypes)

TIMESTAMP       datetime64[ns]
RECORD                 float64
Bat_Avg                float64
Pit_WL_Avg             float64
L_UR_Avg               float64
L_UL_Avg               float64
L_DR_Avg               float64
L_DL_Avg               float64
Load_Avg               float64
WL_FMR_Avg             float64
vel_P_Tot              float64
turb1000_Avg           float64
turb4000_Avg           float64
DiffVolt_Avg           float64
hp_Tot(1)              float64
hp_Tot(2)              float64
hp_Tot(3)              float64
hp_Tot(4)              float64
hp_Tot(5)              float64
hp_Tot(6)              float64
hp_Tot(7)              float64
hp_Tot(8)              float64
hp_Tot(9)              float64
hp_Tot(10)             float64
dtype: object


In [55]:
# Fill out missing date&time
df2 = df1.set_index('TIMESTAMP').asfreq('T', fill_value=0).reset_index()
#Fill out NaNs with 0.
df3 = df2.fillna(0)
print(df3)

                 TIMESTAMP    RECORD  Bat_Avg  Pit_WL_Avg  L_UR_Avg  L_UL_Avg  \
0      2018-03-13 16:23:00     135.0    12.71       11.55     57.07     11.91   
1      2018-03-13 16:24:00     136.0    12.71       11.56     57.07     11.91   
2      2018-03-13 16:25:00     137.0    12.71       11.57     57.01     11.91   
3      2018-03-13 16:26:00     138.0    12.72       11.58     57.06     11.91   
4      2018-03-13 16:27:00     139.0    12.72       11.57     57.06     11.91   
5      2018-03-13 16:28:00     140.0    12.72       11.59     57.05     11.91   
6      2018-03-13 16:29:00     141.0    12.72       11.58     57.04     11.91   
7      2018-03-13 16:30:00     142.0    12.72       11.59     57.05     11.91   
8      2018-03-13 16:31:00     143.0    12.72       11.61     57.05     11.91   
9      2018-03-13 16:32:00     144.0    12.72       11.60     57.00     11.91   
10     2018-03-13 16:33:00     145.0    12.72       11.61     57.03     11.91   
11     2018-03-13 16:34:00  

In [56]:
df4 = df3.set_index('TIMESTAMP').resample('10T').mean().reset_index()
print(df4)


                TIMESTAMP    RECORD    Bat_Avg  Pit_WL_Avg    L_UR_Avg  \
0     2018-03-13 16:20:00     138.0  12.715714   11.571429   57.051429   
1     2018-03-13 16:30:00     146.5  12.720000   11.611000   57.030000   
2     2018-03-13 16:40:00     156.5  12.720000   11.637000   56.990000   
3     2018-03-13 16:50:00     166.5  12.720000   11.650000   56.932000   
4     2018-03-13 17:00:00     176.5  12.718000   11.658000   56.939000   
5     2018-03-13 17:10:00     186.5  12.719000   11.658000   56.899000   
6     2018-03-13 17:20:00     196.5  12.720000   11.660000   56.846000   
7     2018-03-13 17:30:00     206.5  12.720000   11.663000   56.873000   
8     2018-03-13 17:40:00     216.5  12.720000   11.670000   56.866000   
9     2018-03-13 17:50:00     226.5  12.724000   11.685000   56.899000   
10    2018-03-13 18:00:00     236.5  12.721000   11.713000   56.854000   
11    2018-03-13 18:10:00     246.5  12.724000   11.743000   56.786000   
12    2018-03-13 18:20:00     256.5  1

In [57]:
#Import Rainfall, WL, Velocity data from another file.
#Planning to get this data and "df4" together.

#Sort the data in the file
#read a file
fileR = open('Rainfall_WL_Velocity_2018.txt')
AllfileR = fileR.readlines()
file.close()
AllfilelistedR = [0]*len(AllfileR)

#split each list at blanks
for n in range(len(AllfileR)):
    AllfilelistedR[n] = AllfileR[n].split()

DataR = AllfilelistedR[1:]
DatatitleR = AllfilelistedR[0][:]


In [58]:
#Start using pandas
dfR = pd.DataFrame(DataR, dtype=np.float64)
#Put date and time together because I wanna let "to_datetime" read date and time later.
date_timeR = dfR[:][0] + " " + dfR[:][1]
#Let "to_datatime" read the date and time and convert them into datetime64
converted_timeR = pd.to_datetime(date_timeR)
# print(converted_time)
#insert this date&time data into the Data set(Delite the original date and time(Column 0 & 1))
df1R = pd.concat([converted_timeR, dfR.iloc[:, 2:]], axis=1)
#Put titles on the columns
df1R.columns = ["TIME","Rainfall","Velocity","WL"]
print(df1R)

                     TIME  Rainfall  Velocity   WL
0     2018-01-01 00:00:00       0.0      0.65  1.4
1     2018-01-01 00:10:00       0.0      0.65  1.4
2     2018-01-01 00:20:00       0.0      0.65  1.3
3     2018-01-01 00:30:00       0.0      0.65  1.3
4     2018-01-01 00:40:00       0.0      0.65  1.4
5     2018-01-01 00:50:00       0.0      0.65  1.3
6     2018-01-01 01:00:00       0.0      0.65  1.3
7     2018-01-01 01:10:00       0.0      0.65  1.4
8     2018-01-01 01:20:00       0.0      0.65  1.4
9     2018-01-01 01:30:00       0.0      0.65  1.4
10    2018-01-01 01:40:00       0.0      0.65  1.3
11    2018-01-01 01:50:00       0.0      0.65  1.4
12    2018-01-01 02:00:00       0.0      0.65  1.4
13    2018-01-01 02:10:00       0.0      0.65  1.3
14    2018-01-01 02:20:00       0.0      0.65  1.3
15    2018-01-01 02:30:00       0.0      0.65  1.3
16    2018-01-01 02:40:00       0.0      0.65  1.3
17    2018-01-01 02:50:00       0.0      0.65  1.3
18    2018-01-01 03:00:00      

In [59]:
# Fill out missing date&time with the mean values of each column.(Notice!! I haven't checked the filled out data. You should check out if they are filled out correctly)
df2R = df1R.set_index('TIME').asfreq('10T', fill_value= 0).reset_index()

In [60]:
#Output the date of the first and the last data
#And then put cut out df1R data by using these date
print(df4["TIMESTAMP"].head(1))
print(df4["TIMESTAMP"].tail(1))

print(df1R[df2R['TIME'] == '2018-03-13 16:20:00'])
print(df1R[df2R['TIME'] == '2018-12-16 16:20:00'])


df3R = df2R[10322:50354+1].reset_index(drop=True)
print(df3R)

0   2018-03-13 16:20:00
Name: TIMESTAMP, dtype: datetime64[ns]
40032   2018-12-16 16:20:00
Name: TIMESTAMP, dtype: datetime64[ns]
                     TIME  Rainfall  Velocity   WL
10322 2018-03-13 16:20:00       0.0      0.65  0.7
                     TIME  Rainfall  Velocity   WL
50354 2018-12-18 18:10:00       0.0      1.97  4.9
                     TIME  Rainfall  Velocity   WL
0     2018-03-13 16:20:00       0.0      0.65  0.7
1     2018-03-13 16:30:00       0.0      0.65  0.8
2     2018-03-13 16:40:00       0.0      0.65  0.7
3     2018-03-13 16:50:00       0.0      0.65  0.7
4     2018-03-13 17:00:00       0.0      0.65  0.7
5     2018-03-13 17:10:00       0.0      0.65  0.8
6     2018-03-13 17:20:00       0.0      0.65  0.8
7     2018-03-13 17:30:00       0.0      0.65  0.8
8     2018-03-13 17:40:00       0.0      0.65  0.8
9     2018-03-13 17:50:00       0.0      0.65  0.8
10    2018-03-13 18:00:00       0.0      0.65  0.8
11    2018-03-13 18:10:00       0.0      0.65  0.8
12 

C:\Users\yutak\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\yutak\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [61]:
df_fin = pd.concat([df4, df3R.iloc[:, 1:]], axis=1)
print(df_fin)

                TIMESTAMP    RECORD    Bat_Avg  Pit_WL_Avg    L_UR_Avg  \
0     2018-03-13 16:20:00     138.0  12.715714   11.571429   57.051429   
1     2018-03-13 16:30:00     146.5  12.720000   11.611000   57.030000   
2     2018-03-13 16:40:00     156.5  12.720000   11.637000   56.990000   
3     2018-03-13 16:50:00     166.5  12.720000   11.650000   56.932000   
4     2018-03-13 17:00:00     176.5  12.718000   11.658000   56.939000   
5     2018-03-13 17:10:00     186.5  12.719000   11.658000   56.899000   
6     2018-03-13 17:20:00     196.5  12.720000   11.660000   56.846000   
7     2018-03-13 17:30:00     206.5  12.720000   11.663000   56.873000   
8     2018-03-13 17:40:00     216.5  12.720000   11.670000   56.866000   
9     2018-03-13 17:50:00     226.5  12.724000   11.685000   56.899000   
10    2018-03-13 18:00:00     236.5  12.721000   11.713000   56.854000   
11    2018-03-13 18:10:00     246.5  12.724000   11.743000   56.786000   
12    2018-03-13 18:20:00     256.5  1

In [62]:
#Save data
df_fin.to_csv(path_or_buf="logger1_2018-03-13_2018-12-16.csv")